In [1]:
import ipywidgets as wg
from conc_bm import ConcreteBeam
import pandas as pd

In [2]:
class ConcBeamLayout(object):
    
    def __init__(self, model=None):
        self.model = model
        self.initialize()
        
    def initialize(self):
        
        # Create tab layout for plotting. Setup plot output widgets. 
        self.plot_box = wg.Tab()
        self.xsect_plot_output = wg.Output()
        self.strain_plot_output = wg.Output()
        self.stress_plot_output = wg.Output()
        self.force_plot_output = wg.Output()
        self.xsect_plot_box = wg.VBox([self.xsect_plot_output])
        self.strain_plot_box = wg.VBox([self.strain_plot_output])
        self.stress_plot_box = wg.VBox([self.stress_plot_output])
        self.force_plot_box = wg.VBox([self.force_plot_output])
        self.plot_box.children = [self.xsect_plot_box, self.strain_plot_box, self.stress_plot_box, self.force_plot_box]
        self.plot_box.set_title(0, 'Section')
        self.plot_box.set_title(1, 'Strain')
        self.plot_box.set_title(2, 'Stress')
        self.plot_box.set_title(3, 'Force')
        
        self.results_output = wg.Output()
        
        self.long_reinf = []
        
        self.input_box = self.build_input_box()
        
        # Initialize calc.
        self.xsect_shape.calc()
        self.material.calc()
        for b in self.long_reinf:
            b.calc()
        self.model.calc()
        
        # Create initial plot.
        self.plot_xsect()
        
    def build_input_box(self):
        
        calc_btn = wg.Button(description='Run Calc', layout=wg.Layout(width='auto'))
        calc_btn.on_click(self.calc_plot)
        
        self.applied_load = self.new_applied_load()
        self.xsect_shape = self.new_xsect_shape()
        self.material = self.new_material()
        
        # Load input box. 
        applied_load_input = wg.VBox([self.applied_load.Mu, self.applied_load.Vu])
        
        # Concrete inputs box.
        conc_input = wg.VBox([wg.Label(value='Section Geometry'), self.xsect_shape.b, self.xsect_shape.h, 
                              wg.Label(value='Material Properties'), self.material.fc_prime, self.material.strain_ult, self.material.Ec])

        # Create longitudinal steel new layer button and table.
        new_long_reinf_btn = wg.Button(description='New Bar Layer')
        new_long_reinf_btn.on_click(self.new_long_reinf)
        long_reinf_tbl = self.long_reinf_tbl()
        steel_input = wg.VBox([new_long_reinf_btn, long_reinf_tbl])
        
        # Build the input layout.
        applied_load_input_box = wg.Accordion(children=[applied_load_input])
        applied_load_input_box.set_title(0, 'Applied Loads')
        conc_input_box = wg.Accordion(children=[conc_input])
        conc_input_box.set_title(0, 'Concrete')
        steel_input_box = wg.Accordion(children=[steel_input])
        steel_input_box.set_title(0, 'Steel Reinforcement')
        # Consolidate all input into one box.
        input_box = wg.VBox([applied_load_input_box, conc_input_box, steel_input_box, calc_btn])
    
        # Create one set of bars.
        self.new_long_reinf(None)
        
        return input_box
        
    def calc_plot(self, b):
        # Regenerate plot.
        self.plot_all()
        
        # Calc all.
        self.xsect_shape.calc()
        self.material.calc()
        for b in self.long_reinf:
            b.calc()
        self.model.calc()
        
        self.results_output.clear_output()
        # Create output.
        bar_cols = ['size', 'qty', 'x', 'y', r'$\epsilon_{s}$', r'$\sigma_{s}$', 'F' ]
        bar_data = {}
        for b in self.model.long_reinf:
            bar_data[b.name] = [b.size, b.qty, b.x, b.y, b.strain, b.stress, b.F]
        bars_df = pd.DataFrame.from_dict(bar_data, orient='index', columns=bar_cols)
        
        with self.results_output:
            display(wg.HTMLMath("<h3> Bars </h3>"))
            display(bars_df)
            
            print('c =', self.model.c, 'in')
            print('a =', self.model.a, 'in')
            print('beta1 =', self.model.material.beta1)
            print('phi =', self.model.phi)
            print('Mn =', self.model.Mnx, 'lb*in')
            print('phi*Mn =', self.model.phi_Mnx, 'lb*in')
            print('phi*Mn =', self.model.phi_Mnx/12/1000, 'k*ft')
    
    def plot_xsect(self):
        section_fig = self.model.plot_xsect()
        self.xsect_plot_output.clear_output()
        with self.xsect_plot_output:
            section_fig.show()
        
    def plot_all(self):
        # Clear existing plots.
        self.xsect_plot_output.clear_output()
        self.strain_plot_output.clear_output()
        self.stress_plot_output.clear_output()
        self.force_plot_output.clear_output()
        
        # Create new plots.
        section_plot = self.model.plot_xsect()
        self.model.plot_section_analysis()
        strain_plot = self.model.plot_strain()
        stress_plot = self.model.plot_stress()
        force_plot = self.model.plot_force()
        
        # Show new plots.
        with self.xsect_plot_output:
            section_plot.show()            
        with self.strain_plot_output:
            strain_plot.show()
        with self.stress_plot_output:
            stress_plot.show()
        with self.force_plot_output:
            force_plot.show()
                
    def long_reinf_tbl(self):
        # Define table layout properties.
        col_layout = wg.Layout(width='100%')
        
        # Create longitudinal steel table column labels.
        self.name_lbl = wg.Label(value='name')
        self.size_lbl = wg.Label(value='size')
        self.qty_lbl = wg.Label(value='qty.')
        self.x_lbl = wg.Label(value='x')
        self.y_lbl = wg.Label(value='y')
        self.fy_lbl =  wg.Label(value="fy")
        self.Es_lbl =  wg.Label(value="Es")
        self.delete_lbl = wg.Label(value='Delete')
        
        # Create longitudinal steel columns.
        self.name_box = wg.VBox([self.name_lbl], layout=col_layout)
        self.size_box = wg.VBox([self.size_lbl], layout=col_layout)
        self.qty_box = wg.VBox([self.qty_lbl], layout=col_layout)
        self.x_box = wg.VBox([self.x_lbl], layout=col_layout)
        self.y_box = wg.VBox([self.y_lbl], layout=col_layout)
        self.fy_box =  wg.VBox([self.fy_lbl], layout=col_layout)
        self.Es_box =  wg.VBox([self.Es_lbl], layout=col_layout)
        self.delete_box = wg.VBox([self.delete_lbl], layout=col_layout)
        
        # Create longitudinal table.
        long_reinf_tbl = wg.HBox([self.name_box, self.size_box, self.qty_box, self.x_box, self.y_box, self.fy_box, self.Es_box, self.delete_box])
        return long_reinf_tbl

    def new_applied_load(self):
        model = self.model.new_applied_load()
        layout = AppliedLoad(self, model)
        return layout
        
    def new_xsect_shape(self):
        model = self.model.new_xsect_shape()
        layout = XsectShapeLayout(self, model)
        return layout
    
    def new_material(self):
        model = self.model.new_material()
        layout = ConcMaterialLayout(self, model)
        return layout
    
    def new_long_reinf(self, b):
        # Create the new bar layer model and layout. Give the model to the new bar layer layout and add the layout to the 
        # concrete beam layout. 
        model = self.model.new_long_reinf()
        layout = LongReinfLayout(self, model)
        self.long_reinf.append(layout)
        self.rebuild_long_reinf_tbl()
        self.plot_xsect()
        
    def delete_long_reinf(self, long_reinf):
        # Remove the bar from the list of bars in both the model and layout and rebuild the bar layer table.
        self.model.long_reinf.remove(long_reinf.model)
        self.long_reinf.remove(long_reinf)
        self.rebuild_long_reinf_tbl()
        
    def rebuild_long_reinf_tbl(self):
        # Distribute bar info into columns.
        self.name_box.children = tuple([self.name_lbl] + [b.name for b in self.long_reinf])
        self.size_box.children = tuple([self.size_lbl] + [b.size for b in self.long_reinf])
        self.qty_box.children = tuple([self.qty_lbl] + [b.qty for b in self.long_reinf])
        self.x_box.children = tuple([self.x_lbl] + [b.x for b in self.long_reinf])
        self.y_box.children = tuple([self.y_lbl] + [b.y for b in self.long_reinf])
        self.fy_box.children = tuple([self.fy_lbl] + [b.fy for b in self.long_reinf])
        self.Es_box.children = tuple([self.Es_lbl] + [b.Es for b in self.long_reinf])
        self.delete_box.children = tuple([self.delete_lbl] + [b.delete_btn for b in self.long_reinf])


In [3]:
class AppliedLoad(object):
    
    def __init__(self, parent, model):
        self.parent = parent
        self.model = model
        
        cell_layout = wg.Layout(width='auto')

        self.Mu = wg.FloatText(value=model.Mu, description=r'$M_{u}$ &nbsp <i>[lb*in]</i>', layout=cell_layout)
        self.Vu = wg.FloatText(value=model.Vu, description=r'$V_{u}$ &nbsp <i>[lb]</i>', layout=cell_layout)
        
    def calc(self):
        self.model.calc(Mu=self.Mu.value, Vu=self.Vu.value)

In [4]:
class ConcMaterialLayout(object):
    
    def __init__(self, parent, model):
        self.parent = parent
        self.model = model
        
        cell_layout = wg.Layout(width='auto')

        self.fc_prime = wg.FloatText(value=model.fc_prime, description=r'$f^{\prime}_{c}$ &nbsp <i>[psi]</i>', layout=cell_layout)
        self.strain_ult = wg.FloatText(value=model.strain_ult, description=r'$\epsilon_{u}$ &nbsp <i>[in/in]</i>', layout=cell_layout)
        self.Ec = wg.FloatText(value=model.Ec, description=r'$E_c$ &nbsp <i>[psi]</i>', layout=cell_layout)
        
    def calc(self):
        self.model.calc(fc_prime=self.fc_prime.value, strain_ult=self.strain_ult.value, Ec=self.Ec.value)

In [5]:
class XsectShapeLayout(object):
    
    def __init__(self, parent, model):
        self.parent = parent
        self.model = model
        
        cell_layout = wg.Layout(width='auto')

        self.b = wg.FloatText(value=model.b, description=r'$b$ &nbsp <i>[in]</i>', layout=cell_layout)
        self.h = wg.FloatText(value=model.h, description=r'$h$ &nbsp <i>[in]</i>', layout=cell_layout)
        
        self.b.observe(self.plot_update, names='value')
        self.h.observe(self.plot_update, names='value')
    
    def plot_update(self, change):
        self.calc()
        self.parent.plot_xsect()
        
    def calc(self):
        self.model.calc(b=self.b.value, h=self.h.value)

In [6]:
class LongReinfLayout(object):
    
    def __init__(self, parent, model):
        self.parent = parent
        self.model = model
        
        cell_layout = wg.Layout(width='auto')
        
        self.name = wg.IntText(value=model.name, layout=cell_layout, disabled=True)
        self.size = wg.Dropdown(options=['3', '4', '5', '6', '7', '8', '9', '10', '11', '14', '18'], value=model.size, layout=cell_layout)
        self.qty = wg.FloatText(value=model.qty, layout=cell_layout)
        self.x = wg.FloatText(value=model.x, layout=cell_layout)
        self.y = wg.FloatText(value=model.y, layout=cell_layout)
        self.fy = wg.FloatText(value=model.fy, layout=cell_layout)
        self.Es = wg.FloatText(value=model.Es, layout=cell_layout)
        
        self.name.observe(self.plot_update, names='value')
        self.size.observe(self.plot_update, names='value')
        self.qty.observe(self.plot_update, names='value')
        self.x.observe(self.plot_update, names='value')
        self.y.observe(self.plot_update, names='value')
        
        self.delete_btn = wg.Button(description='Delete', layout=cell_layout)
        self.delete_btn.on_click(self.delete)
    
    def plot_update(self, change):
        self.calc()
        self.parent.plot_xsect()
        
    def delete(self, b):
        # Delete the model and remove the layout.
        self.parent.delete_long_reinf(self)
        
    def calc(self):
        self.model.calc(size=self.size.value, qty=self.qty.value, x=self.x.value, y=self.y.value, fy=self.fy.value, Es=self.Es.value)

# Concrete Beam Section Design <v0.1>

## Introduction:

Concrete beam design.

### Assumptions and Limitations:
- Only uniaxial bending capacity is calculated. Axial loads and moments about the y-axis are not supported.

In [7]:
from general_app_layout import AppLayout, StackedLayout
beam = ConcBeamLayout(ConcreteBeam())
app = StackedLayout(beam.input_box, beam.plot_box, beam.results_output)
app.main_view